# Deep Learning for cellular image

In [1]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3070 Laptop GPU'

In [2]:
import urllib
import numpy as np

In [3]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained= True)

Using cache found in C:\Users\abhis/.cache\torch\hub\pytorch_vision_v0.10.0


In [4]:
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

# Transfer learning using torch

In [4]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
# stride controls the stride for the cross-correlation, a single number or a tuple.
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda', index=0)

# Testing on CNS data

In [10]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
}

In [11]:
img_dir = 'E:/Abhi/Alienware/INDIANA UNIVERSITY/CNS Research test data'

In [12]:
img_data = {x: datasets.ImageFolder(os.path.join(img_dir, x), data_transforms[x]) for x in ['train','test']}

In [13]:
img_data

{'train': Dataset ImageFolder
     Number of datapoints: 1200
     Root location: E:/Abhi/Alienware/INDIANA UNIVERSITY/CNS Research test data\train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(299, 299), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'test': Dataset ImageFolder
     Number of datapoints: 600
     Root location: E:/Abhi/Alienware/INDIANA UNIVERSITY/CNS Research test data\test
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(299, 299), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [14]:
dataloaders = {x: torch.utils.data.DataLoader(img_data[x], batch_size =4, shuffle= True, num_workers =4) for x in ['train','test']}

In [15]:
img_sizes = {x: len(img_data[x]) for x in ['train','test']}

In [16]:
class_names = img_data['train'].classes

In [17]:
class_names

['colon',
 'endometrium_1',
 'endometrium_2',
 'kidney',
 'liver',
 'lung',
 'lymph_node',
 'pancreas',
 'skin_1',
 'skin_2',
 'small_intestine',
 'spleen']

In [25]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print("-"*10)
        
        # each epoch has a training and validation phase
        for phase in ['train','test']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            running_corrects = 0
            
            # iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels=labels.to(device)
                
                # zero the optimizer gradients
                optimizer.zero_grad()
                
                
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs,1)
                    loss = criterion(outputs, labels)
#                     loss2 = criterion(aux_outs, labels)
#                     loss = loss1+0.4*loss2
                    
                    # backward+optimize
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                
                
                running_loss+= loss.item()*inputs.size(0)
                running_corrects+= torch.sum(preds==labels.data)
                
            if phase=='train':
                scheduler.step()
                    
            epoch_loss = running_loss/img_sizes[phase]
            epoch_acc = running_corrects.double()/img_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase=='test' and epoch_acc>best_acc:
                best_acc=epoch_acc
                best_model_wts=copy.deepcopy(model.state_dict())
                
        
    print('Best test Acc:{:2f}'.format(best_acc))
        
    model.load_state_dict(best_model_wts)
    return model

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained= True)
model.aux_logits=False

num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 12.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 12)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Using cache found in C:\Users\abhis/.cache\torch\hub\pytorch_vision_v0.10.0


In [21]:
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=30)

Epoch 0/29
----------
train Loss: 2.4457 Acc: 0.1200
test Loss: 2.2820 Acc: 0.3667
Epoch 1/29
----------
train Loss: 2.2374 Acc: 0.2908
test Loss: 2.0439 Acc: 0.5133
Epoch 2/29
----------
train Loss: 2.0366 Acc: 0.3758
test Loss: 1.7918 Acc: 0.5467
Epoch 3/29
----------
train Loss: 1.7969 Acc: 0.4658
test Loss: 1.5094 Acc: 0.6150
Epoch 4/29
----------
train Loss: 1.5935 Acc: 0.5125
test Loss: 1.3268 Acc: 0.6117
Epoch 5/29
----------
train Loss: 1.4729 Acc: 0.5275
test Loss: 1.1452 Acc: 0.6817
Epoch 6/29
----------
train Loss: 1.3596 Acc: 0.5750
test Loss: 0.9908 Acc: 0.7000
Epoch 7/29
----------
train Loss: 1.2735 Acc: 0.6058
test Loss: 0.9187 Acc: 0.7283
Epoch 8/29
----------
train Loss: 1.2420 Acc: 0.6092
test Loss: 0.9656 Acc: 0.7117
Epoch 9/29
----------
train Loss: 1.2454 Acc: 0.6067
test Loss: 0.9314 Acc: 0.7267
Epoch 10/29
----------
train Loss: 1.2656 Acc: 0.5933
test Loss: 0.9209 Acc: 0.7417
Epoch 11/29
----------
train Loss: 1.2066 Acc: 0.6292
test Loss: 0.8992 Acc: 0.7500
Ep

In [22]:
# 77.50 % test accuracy achieved on test dataset #run1 with inception_v3 pretrained
# 76.50 % test accuracy achieved on test dataset #run2 with inception_v3 pretrained

# CNN from scratch

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
}

In [8]:
img_dir = 'E:/Abhi/Alienware/INDIANA UNIVERSITY/CNS Research test data'

In [9]:
img_data = {x: datasets.ImageFolder(os.path.join(img_dir, x), data_transforms[x]) for x in ['train','test']}

In [10]:
dataloaders = {x: torch.utils.data.DataLoader(img_data[x], batch_size =4, shuffle= True, num_workers =4) for x in ['train','test']}

In [11]:
img_sizes = {x: len(img_data[x]) for x in ['train','test']}

In [12]:
img_sizes

{'train': 1200, 'test': 600}

In [34]:
# 1st trial
class myCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 4, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 16, 5)
        self.fc1 = nn.Linear(80656,512 )
        self.fc2 = nn.Linear(512, 64)
        self.fc3 = nn.Linear(64, 12)
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [47]:
#2nd trial
class myCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        self.conv3= nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        self.fc1 = nn.Linear(57600, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 12)
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [35]:
model = myCNN()

In [37]:
# torch.cuda.empty_cache()

In [38]:
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [39]:
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 2.4852 Acc: 0.0725
test Loss: 2.4821 Acc: 0.0850
Epoch 1/24
----------
train Loss: 2.4794 Acc: 0.0792
test Loss: 2.4761 Acc: 0.0950
Epoch 2/24
----------
train Loss: 2.4720 Acc: 0.0867
test Loss: 2.4673 Acc: 0.1033
Epoch 3/24
----------
train Loss: 2.4614 Acc: 0.0958
test Loss: 2.4571 Acc: 0.0933
Epoch 4/24
----------
train Loss: 2.4452 Acc: 0.1125
test Loss: 2.4417 Acc: 0.1017
Epoch 5/24
----------
train Loss: 2.4244 Acc: 0.1142
test Loss: 2.4198 Acc: 0.1117
Epoch 6/24
----------
train Loss: 2.3907 Acc: 0.1500
test Loss: 2.3820 Acc: 0.1400
Epoch 7/24
----------
train Loss: 2.3573 Acc: 0.1742
test Loss: 2.3769 Acc: 0.1533
Epoch 8/24
----------
train Loss: 2.3489 Acc: 0.1850
test Loss: 2.3676 Acc: 0.1467
Epoch 9/24
----------
train Loss: 2.3423 Acc: 0.1967
test Loss: 2.3554 Acc: 0.1650
Epoch 10/24
----------
train Loss: 2.3334 Acc: 0.2050
test Loss: 2.3549 Acc: 0.1683
Epoch 11/24
----------
train Loss: 2.3264 Acc: 0.2008
test Loss: 2.3472 Acc: 0.1733
Ep

In [ ]:
# 1st trial 1st run gave 30% test accuracy
# it gave 19% accuracy in the 2nd run

In [50]:
# MODEL WITH RESIZED 499 #2nd trial
model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 2.4842 Acc: 0.0858
test Loss: 2.4805 Acc: 0.1133
Epoch 1/24
----------
train Loss: 2.4795 Acc: 0.1000
test Loss: 2.4758 Acc: 0.1017
Epoch 2/24
----------
train Loss: 2.4728 Acc: 0.1117
test Loss: 2.4668 Acc: 0.1233
Epoch 3/24
----------
train Loss: 2.4619 Acc: 0.1225
test Loss: 2.4518 Acc: 0.1517
Epoch 4/24
----------
train Loss: 2.4436 Acc: 0.1450
test Loss: 2.4211 Acc: 0.1567
Epoch 5/24
----------
train Loss: 2.3973 Acc: 0.1683
test Loss: 2.3632 Acc: 0.1700
Epoch 6/24
----------
train Loss: 2.3207 Acc: 0.1867
test Loss: 2.2913 Acc: 0.1917
Epoch 7/24
----------
train Loss: 2.2554 Acc: 0.1992
test Loss: 2.2628 Acc: 0.2050
Epoch 8/24
----------
train Loss: 2.2161 Acc: 0.2075
test Loss: 2.2476 Acc: 0.1933
Epoch 9/24
----------
train Loss: 2.2022 Acc: 0.2175
test Loss: 2.2491 Acc: 0.1967
Epoch 10/24
----------
train Loss: 2.2126 Acc: 0.2150
test Loss: 2.2233 Acc: 0.2033
Epoch 11/24
----------
train Loss: 2.1950 Acc: 0.2083
test Loss: 2.2199 Acc: 0.2100
Ep

## Q.Explain why some images might have been classified incorrectly.

<div class="alert alert-info" style="background-color:#006a79; color:white; padding:0px 10px; border-radius:5px;">
    <p>-->first the model relates the test data with training data mathematically.so it can happen that the mathematical correlation is absolutely correct all the time.</p><br>
    <p>-->As the pixels for the images are very large when we crop the images for preprocessing it might happen the image can loose necessary informations.</p><br>
    <p>-->The number of layers not built in as exact solution. Even if we tuned the inception_v3 dataset we could not achieve above 90% accuracy as model was unable to generalize even if the inception_v3 as more than 100 layers.</p><br>
</div>